# Getting the info that the client needs for D3.js

Often it is easier to transform data in the backend to be used by D3. In general all data in D3 needs to be a list of dicts. In this example I'm using an existing `world` object to demonstrate the methods that do these transformations. 

In [1]:
#I'm mapping to the actual files in the repo so that I can also use this to troubleshoot
import os
import sys
import numpy as np
import pandas as pd
import pickle
import django
sys.path.append('../..')
#Loading my project settings from prodweb. This allows me to load and query models. 
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'prodweb.settings')
django.setup()


with (open("../pickles/billmanhworld.pkl", 'rb')) as pickle_file:
    world = pickle.load(pickle_file)

In [2]:
sys.path.append('..')
from lib.builders import world as w

`w.get_area_data(world)` gets the area relevant to where the character is. This includes surrounding areas.

In [3]:
world

In [4]:
print(world.Character.location)
pd.DataFrame(w.get_area_data(world)).T

[2, 30]


,aware,danger,elevation,feature,key,nation,nation number,rainfall,terrain,turn_last_visited,visited,x,y
area,0,0.4,11,Cambridge,2:30,Snowring,2,11,town,0,1,2,30
NArea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,void,NaN,NaN,NaN,NaN
SArea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,void,NaN,NaN,NaN,NaN
EArea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,void,NaN,NaN,NaN,NaN
WArea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,void,NaN,NaN,NaN,NaN


For other game principals, it's easy to just make regular `pandas` and `numpy` queries on those datasets. 

In [5]:
world.df_features.loc[world.Character.get_location_key(),'visited'] >= 1

True

Querying the terrain is easy as well. For example, here is a query of all of the world where the _terrain_ is _desert_ and the _danger_ is above .5. These queries are fast and cheap. 

In [6]:
world.df_features.loc[(world.df_features['terrain']=='desert') & 
                     (world.df_features['danger']>=.5)].head()

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
24:11,11,1.0,24,24:11,16.0,desert,NaN,1,Malvale,0,0,0,0.890
27:11,11,1.0,27,27:11,8.0,desert,NaN,1,Malvale,0,0,0,0.522
30:11,11,1.0,30,30:11,6.0,desert,NaN,1,Malvale,0,0,0,0.847
24:12,12,0.0,24,24:12,22.0,desert,NaN,1,Malvale,0,0,0,0.673
26:12,12,0.0,26,26:12,12.0,desert,NaN,1,Malvale,0,0,0,0.732


You can also _mask_ the terrain where the character has no knowledge. 

In [7]:
masked_world = w.mask_unknown(world)
world.Character.title = "Noble"
masked_world.head()

,y,rainfall,x,key,elevation,terrain,feature,nation number,nation,visited,aware,turn_last_visited,danger
key,,,,,,,,,,,,,
0:0,0,9.0,0,0:0,0.0,ocean,NaN,4,Snowblaze,0,1,0,0.184
1:0,0,10.0,1,1:0,2.0,ocean,NaN,4,Snowblaze,0,1,0,0.289
2:0,0,13.0,2,2:0,2.0,ocean,NaN,4,Snowblaze,0,1,0,-0.107
3:0,0,17.0,3,3:0,-1.0,ocean,NaN,4,Snowblaze,0,1,0,0.263
4:0,0,22.0,4,4:0,-1.0,ocean,NaN,4,Snowblaze,0,1,0,0.435


## Analysis via list comprehension

Querying people is also easy, but you need to use _list comprehension_ to do it. This is the general structure. 

In [8]:
[[person.name for person in t.population] for t in world.towns]

[['Elissove', 'Theklipak', 'Lackrnan', 'Defrep', 'Shosalkath'],
 ['Quigennis', 'Aloreshent', 'Pifaddab', 'Zhahorkess'],
 ['Marrath', 'Xallad', 'Elobrat'],
 ['Xarnust', 'Bropproh', 'Shosalkur'],
 ['Elekrackep', 'Quanetteh'],
 ['Illiahilloe', 'Kilsot', 'Kindod', 'Behorka'],
 ['Soklipep', 'Quakillan', 'Alowil'],
 ['Olspove', 'Sheldosh'],
 ['Xibrane', 'Quokrackelton'],
 ['Mirpan', 'Pepeltun'],
 ['Vasassoe', 'Thegrish', 'Zhobrett'],
 ['Tafillur', 'Tholnodin', 'Drondaf'],
 ['Truranda', 'Kirpart', 'Alscir'],
 ['Parushunt', 'Noppsodin', 'Chiladdis'],
 ['Ozennea', 'Xiyellish'],
 ['Kannis', 'Stupartis'],
 ['Ulrushic', 'Dowildumber'],
 ['Senettnan'],
 ['Irareshnan']]

Query the people in the town by setting conditions in the list. These people are the highest in temperment. 

In [9]:
[[(person.name,person.temperment) for person in t.population if person.temperment>.5] for t in world.towns]

[[('Elissove', 0.54),
  ('Theklipak', 0.91),
  ('Lackrnan', 0.71),
  ('Shosalkath', 0.64)],
 [('Aloreshent', 0.77), ('Pifaddab', 0.94), ('Zhahorkess', 0.67)],
 [('Xallad', 0.94)],
 [('Bropproh', 0.58), ('Shosalkur', 0.83)],
 [('Elekrackep', 0.67)],
 [('Illiahilloe', 0.95), ('Kilsot', 0.66), ('Kindod', 1.0), ('Behorka', 0.72)],
 [('Soklipep', 0.82)],
 [],
 [('Quokrackelton', 0.61)],
 [('Pepeltun', 0.85)],
 [('Vasassoe', 0.93), ('Thegrish', 0.83), ('Zhobrett', 0.51)],
 [('Tafillur', 0.86), ('Tholnodin', 0.76)],
 [('Truranda', 0.72), ('Kirpart', 0.62)],
 [('Parushunt', 0.54), ('Chiladdis', 0.84)],
 [('Ozennea', 0.9), ('Xiyellish', 0.59)],
 [('Kannis', 0.61)],
 [('Ulrushic', 0.96), ('Dowildumber', 0.81)],
 [],
 [('Irareshnan', 0.86)]]

You can also get summary stats on towns. For example, the average temperment per town.

In [10]:
[(t.name,np.mean([person.temperment for person in t.population])) for t in world.towns]

[('Starlair', 0.632),
 ('Tearhenge', 0.6575),
 ('Snakespike', 0.36999999999999994),
 ('Demonsforth', 0.5233333333333333),
 ('Soilloch', 0.43500000000000005),
 ('Rockmore', 0.8325),
 ('Cambridge', 0.5299999999999999),
 ('Fencoil', 0.12000000000000001),
 ('Rocklight', 0.495),
 ('Coreloch', 0.45),
 ('Hellcraft', 0.7566666666666667),
 ('Lionblaze', 0.5933333333333334),
 ('Waterridge', 0.6033333333333333),
 ('Fairfold', 0.52),
 ('Fenring', 0.745),
 ('Malhail', 0.48),
 ('Rockhenge', 0.885),
 ('Badgervale', 0.08),
 ('Lionkeep', 0.86)]

For some purposes you may want to get a collection of people and edit them in bulk. This is also possible using list comprehension. 

In [11]:
w.get_people_where_char_has_visited(world)

[{'town': capitol of Cambridge: population: 3 location: [2,30] founded 1006,
  'people': [Soklipep the Speaker of Cambridge,
   Quakillan the commoner,
   Alowil the commoner]}]

In [12]:
r = w.get_people_where_char_has_visited(world)[0]
r

{'town': capitol of Cambridge: population: 3 location: [2,30] founded 1006,
 'people': [Soklipep the Speaker of Cambridge,
  Quakillan the commoner,
  Alowil the commoner]}

In [13]:
r['town'].nation

'Cambridge'

In [14]:
w.get_relationships_node_map(world)

[{'name': 'Cambridge',
  'title': 'capitol of Cambridge',
  'nation': 'Cambridge',
  'population': 3,
  'type': 'capitol',
  'location': '2:30',
  'founded year': 1006,
  'children': [{'name': 'Soklipep',
    'role': 'Speaker of Cambridge',
    'temperment': 0.82,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Quakillan',
    'role': 'commoner',
    'temperment': 0.34,
    'loyalty': 0.5,
    'type': 'person'},
   {'name': 'Alowil',
    'role': 'commoner',
    'temperment': 0.43,
    'loyalty': 0.5,
    'type': 'person'}]}]

## Getting map data (mapData): 

In [28]:
def coordkey(coord):
    '''
    takes a coord [1,1], returns key "1:1"
    '''
    key = ":".join([str(i) for i in coord])
    return key


def get_features_or_NA(world,coord):
    try:
        l = world.df_features.loc[
                chordKey([
                    int(coord[0]),int(coord[1])
                    ])
                ].fillna("none").to_dict(),
    except: 
        l = [{"terrain":"void"}]
    return l[0]


def get_area_data(world):
    """
    note: requires Character
    """
    mapData = {}
    coord = world.Character.location
    key = world.Character.get_location_key()
    l = world.df_features.loc[key]
    mapData = {'area':world.df_features.loc[key].fillna("none").to_dict(),
              'NArea':get_features_or_NA(world,[coord[0],coord[1]-1]),
              'SArea':get_features_or_NA(world,[coord[0],coord[1]+1]),
              'EArea':get_features_or_NA(world,[coord[0]+1,coord[1]]),
              'WArea':get_features_or_NA(world,[coord[0]-1,coord[1]])}
    return mapData


get_area_data(world)

{'area': {'y': 30,
  'rainfall': 11.0,
  'x': 2,
  'key': '2:30',
  'elevation': 11.0,
  'terrain': 'town',
  'feature': 'Cambridge',
  'nation number': 2,
  'nation': 'Snowring',
  'visited': 1,
  'aware': 0,
  'turn_last_visited': 0,
  'danger': 0.4},
 'NArea': {'terrain': 'void'},
 'SArea': {'terrain': 'void'},
 'EArea': {'terrain': 'void'},
 'WArea': {'terrain': 'void'}}

In [31]:
world.df_features.loc[
                chordKey([
                    int(coord[0]),int(coord[1])
                    ])
                ]

NameError: name 'chordKey' is not defined

In [30]:
get_features_or_NA(world,[coord[0],coord[1]-1])

{'terrain': 'void'}

In [ ]:
nations_where_character_has_been = np.unique(where_the_char_has_been['nation'].dropna())
nations_where_character_has_been

You can use this to get lists of people who the character knows. 

In [ ]:
[[t for t in n.get_all_towns(world)] for n in world.nations if n.name in nations_where_character_has_been]

In [ ]:
[[t for t in T.population] for T in world.towns if T.nation in nations_where_character_has_been]

In [ ]:
[(T.nation,T.name) for T in world.towns]

In [ ]:
[[t.get_person_data() for t in T.population] for T in world.towns if T.nation in nations_where_character_has_been]

In [ ]:
masked_world.loc[world.Character.get_location_key()]

You can query nations or towns. `world.nations` is a list of _nations_ and `world.towns` is a list of _towns_

In [ ]:
where_the_char_has_been = world.df_features.loc[(world.df_features['visited']==1)&
                                                (world.df_features['terrain']=='town')]
where_the_char_has_been

## Getting the diplomacy and relationships data for the journal page

In [ ]:
rel = w.get_people_where_char_has_visited(world)
rel

In [ ]:
w.get_relationships_node_map(world)

In [ ]:
hierarchy = [{'nation':i.name,
              'children':i.diplomacy} for i in world.nations]



In [ ]:
p = rel[0]['town'].population[0]


In [ ]:
def get_people_where_char_has_visited(world):
    """
    returns a dictionary of {{"town":[<obj>],"people":[<obj>]}
    filtered to where the character has been. 
    """
    where_the_char_has_been = world.df_features.loc[(world.df_features['visited']==1)&
                                                    (world.df_features['terrain']=='town')].dropna()
    towns_and_people = [{"town":T,
                         "people":[t for t in T.population]} 
                        for T in world.towns if T.name in np.unique(w.where_the_char_has_been['feature'])]
    return towns_and_people

def get_relationships_node_map(world):
    nodes = [{'name':r['town'].name,
              'title':str(r['town']).split(":")[0],
              'nation':r['town'].diplomacy['nation'],
              'population':r['town'].pop,
              'type':r['town'].type,
              'location':r['town'].key,
              'founded year':r['town'].founded,
              'national fealty':r['town'].diplomacy['national fealty'],
             'children': [{'name':p.name,
                          'role':p.role,
                          'temperment':p.temperment,
                          'loyalty':p.loyalty,
                          'type':'person'} for p in r['people']]} 
             for r in w.get_people_where_char_has_visited(world)]
    return nodes
        
w.get_people_where_char_has_visited(world)

In [ ]:
relationships = w.get_relationships_node_map(world)
relationships

In [ ]:
nations = [s['nation'] for s in relationships]
nations

In [ ]:
[i for i in relationships]

In [ ]:
[{'nation':i,'towns':[t for t in relationships if t['nation']==i]} for i in nations]

In [ ]:
[i for i in nations]